In [3]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender
from MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4



#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [4]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')

#Load playlistsIDS
data_playlists= pd.read_csv('all/train.csv',low_memory = False)
userID=data_playlists.playlist_id.unique()

In [5]:
CFrecommender = ItemKNNCFRecommender(URM_all)
CFrecommender.fit()

SLIMrecommender = SLIM_BPR_Cython(URM_all)
SLIMrecommender.fit()

CBFrecommender = ItemKNNCBFRecommender(ICM_all,URM_all)
CBFrecommender.fit()

SLIMErecommender = SLIMElasticNetRecommender(URM_all)
SLIMErecommender.fit()

UCFRecommender = UserKNNCFRecommender(URM_all)
UCFRecommender.fit()

SVDrecommender = PureSVDRecommender (URM_all)
SVDrecommender.fit()

Similarity column 20635 ( 100 % ), 3730.80 column/sec, elapsed time 0.09 min
SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Processed 1211791 ( 100.00% ) in 6.28 seconds. BPR loss is 1.06E+00. Sample per second: 193049
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.22 min
Processed 1211791 ( 100.00% ) in 5.46 seconds. BPR loss is 2.38E+00. Sample per second: 221786
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.30 min
Processed 1211791 ( 100.00% ) in 6.23 seconds. BPR loss is 3.29E+00. Sample per second: 194600
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.40 min
Processed 1211791 ( 100.00% ) in 6.53 seconds. BPR loss is 4.05E+00. Sample per second: 185479
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.51 min
Processed 1211791 ( 100.00% ) in 6.73 seconds. BPR loss is 4.67E+00. Sample per second: 179970
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.61 min
Processed 1211791 ( 100.00% ) in 6.69 seconds. BPR los

SLIM_BPR_Recommender: Epoch 52 of 100. Elapsed time 4.56 min
Processed 1211791 ( 100.00% ) in 5.53 seconds. BPR loss is 1.58E+01. Sample per second: 219276
SLIM_BPR_Recommender: Epoch 53 of 100. Elapsed time 4.64 min
Processed 1211791 ( 100.00% ) in 5.54 seconds. BPR loss is 1.59E+01. Sample per second: 218824
SLIM_BPR_Recommender: Epoch 54 of 100. Elapsed time 4.72 min
Processed 1211791 ( 100.00% ) in 5.06 seconds. BPR loss is 1.61E+01. Sample per second: 239625
SLIM_BPR_Recommender: Epoch 55 of 100. Elapsed time 4.80 min
Processed 1211791 ( 100.00% ) in 4.51 seconds. BPR loss is 1.62E+01. Sample per second: 268785
SLIM_BPR_Recommender: Epoch 56 of 100. Elapsed time 4.87 min
Processed 1211791 ( 100.00% ) in 5.57 seconds. BPR loss is 1.63E+01. Sample per second: 217581
SLIM_BPR_Recommender: Epoch 57 of 100. Elapsed time 4.96 min
Processed 1211791 ( 100.00% ) in 5.32 seconds. BPR loss is 1.64E+01. Sample per second: 227704
SLIM_BPR_Recommender: Epoch 58 of 100. Elapsed time 5.04 min
Pro

In [9]:
#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

import implicit


# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=1000, regularization=0.001,iterations=2)

# train the model on a sparse matrix of item/user/confidence weights
item_user_data=URM_all.T
print("ITEM USER ={}".format(item_user_data.shape))
model.fit(item_user_data)

# recommend items for a user
user_items = item_user_data.T.tocsr()
print("USER ITEM ={}".format(user_items.shape))


#uno per volta predico tutti gli utenti e estraggo dalla lista delle 10 predizioni, dalla tupla, la track
rec_matrix=np.zeros((50446,10))

for u in all_userID:
    rec = model.recommend(u, user_items)
    for x in range (10):
        rec_matrix[u,x]=rec[x][0]
print(rec_matrix)   

ITEM USER =(20635, 50446)


100%|██████████| 2.0/2 [01:31<00:00, 47.62s/it]


USER ITEM =(50446, 20635)
[[15875.  9533. 18735. ... 17016.  4932.  4576.]
 [ 9533. 18356.    23. ... 10015. 19753. 11520.]
 [   26. 15875. 19359. ... 18204. 17599.  3868.]
 ...
 [15875.  3887. 17519. ...  8130.  5207. 10961.]
 [   26. 19359. 12019. ...  9533. 15315. 17306.]
 [15875.    23. 17519. ... 17306. 18356. 11584.]]


In [11]:
#Salviamom la matrice con i ranking per poterlo ibridare in futuro
colonne= 20635
rec_matrix=np.zeros((50446,colonne))

for u in all_userID:
    if (u==1000):
        print("Blocco")
    if (u==5000):
        print("BloccoX")
    if (u==10000):
        print("Blocco1")
    if (u==20000):
        print("Blocco2")
    if (u==30000):
        print("Blocco3")
    if (u==40000):
        print("Blocco4")
    if (u==50000):
        print("Blocco5")
    rec = model.recommend(u, user_items,N=colonne)
    for x in range (len(rec)):
        col=rec[x][0]
        rec_matrix[u,col]=rec[x][1]
print(rec_matrix)  

Blocco
BloccoX
Blocco1
Blocco2
Blocco3
Blocco4
Blocco5
[[-3.07611190e-04 -1.37685775e-03 -3.11374734e-03 ... -6.44895341e-03
  -3.79906059e-03 -7.02443486e-03]
 [ 1.24347359e-02  1.19347572e-02 -8.55531171e-03 ... -2.08741240e-02
   5.68858301e-03  9.80323553e-03]
 [ 1.63063146e-02  1.76310912e-03 -1.37965549e-02 ...  1.19664688e-02
  -2.93323677e-03 -4.07450693e-03]
 ...
 [-2.55913846e-03  5.94201824e-03 -1.61180086e-02 ... -1.89834298e-03
   3.18628992e-03 -4.96131089e-03]
 [ 8.48124735e-03 -2.30362616e-03  6.54044561e-06 ...  6.24435814e-03
   7.87700526e-04  3.61004798e-03]
 [ 1.08573576e-02 -1.47221182e-02  1.43747153e-02 ... -1.02406135e-03
   7.09463656e-03 -3.43429390e-04]]


In [ ]:
from createSubmission import create_submission


SuperSlimSimilarity=ItemKNNSimilarityHybridRecommender(URM_all,SLIMrecommender,SLIMErecommender,"l2")
SuperSlimSimilarity.fit(alpha=0.5, topK=600)

recommender=ItemKNNSimilarityHybridRecommender3(URM_all,SuperSlimSimilarity, CFrecommender,CBFrecommender,"l2")
recommender.fit(topK=600,alpha=0.6,beta=0.2,gamma=0.2)

#create_submission(recommender)

In [22]:
from KNN.ScoresHybridRecommenderXXX import ScoresHybridRecommenderXXX


Hrecommender = ScoresHybridRecommenderXXX(URM_all,SVDrecommender,rec_matrix)
Hrecommender.fit(0.3)

In [23]:
from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender

Finalrecommender = ItemKNNScoresHybridRecommender(URM_all,recommender,Hrecommender)
Finalrecommender.fit(0.85)
create_submission(Finalrecommender)

In [ ]:
recommender_list = [
    #Random,
    #TopPop,
    #P3alphaRecommender,
    #RP3betaRecommender,
    #ItemKNNCFRecommender,
    #UserKNNCFRecommender,
    #MatrixFactorization_BPR_Cython,
    #MatrixFactorization_FunkSVD_Cython,
    #PureSVDRecommender,
    #SLIM_BPR_Cython,
    SLIMElasticNetRecommender
    ]


from Base.Evaluation.Evaluator import SequentialEvaluator

evaluator = SequentialEvaluator(URM_test, [10], exclude_seen=True)


output_root_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_root_path):
    os.makedirs(output_root_path)


logFile = open(output_root_path + "result_all_algorithms.txt", "a")


for recommender_class in recommender_list:

    try:

        print("Algorithm: {}".format(recommender_class))


        recommender = recommender_class(URM_train)
        recommender.fit()
        result=recommender.evaluateRecommendations(URM_test)
        print("Recommender MAP is= {}".format(result["MAP"]))
        #results_run, results_run_string = evaluator.evaluateRecommender(recommender)
       
        #print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_run_string))
        #logFile.write("Algorithm: {}, results: \n{}\n".format(recommender.__class__, results_run_string))
        #logFile.flush()
 
        
    except Exception as e:
        traceback.print_exc()
        logFile.write("Algorithm: {} - Exception: {}\n".format(recommender_class, str(e)))
        logFile.flush()

Algorithm: <class 'SLIM_ElasticNet.SLIMElasticNetRecommender.SLIMElasticNetRecommender'>


#PER ESEGUIRE SUBMISSION
from createSubmission import create_submission

recommender=UserKNNCFRecommender(URM_all)
recommender.fit()

create_submission(recommender)

import re

results_run_string
m = re.search('MAP: (.+?),', results_run_string)
if m:
    found = m.group(1)
found

#PER CONTROLLARI DOPPIONI RACCOMANDAZIONE
racommandation = CFrecommender.recommend(userToPredict)
racommandation

target_playlist= pd.read_csv('all/target_playlists.csv',low_memory = False)
target_playlistIds=target_playlist.playlist_id.values
target_playlistIds.sort()
songs=data_playlists.loc[data_playlists['playlist_id'].isin(target_playlistIds)]
songs

counter=0
for x in target_playlistIds:
        raccomandate=racommandation[counter]
        dfsongs = songs.loc[songs['playlist_id'] == x]
        for row in dfsongs.itertuples():
            if row.track_id in raccomandate:
                print("ERRORRRR")
                print(x)
                print(row.track_id)
        counter=counter+1

